In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from aust_covid.inputs import load_google_mob_year_df
PROJECT_PATH = Path().resolve().parent
DATA_PATH = PROJECT_PATH / 'data'

In [ ]:
locations = [
    'school', 
    'home', 
    'work', 
    'other_locations',
]

In [ ]:
raw_location_matrices = {i: pd.read_csv(DATA_PATH / f'{i}.csv', index_col=0).to_numpy() for i in locations}

In [ ]:
raw_data = pd.concat(
    [
        pd.read_csv(DATA_PATH / '2021_AU_Region_Mobility_Report.csv', index_col=8),
        pd.read_csv(DATA_PATH / '2022_AU_Region_Mobility_Report.csv', index_col=8),
    ]
)
state_data = raw_data.loc[raw_data['sub_region_1'].notnull() & raw_data['sub_region_2'].isnull()]
wa_data = state_data.loc[state_data['sub_region_1'] == 'Western Australia']
non_wa_data = state_data.loc[state_data['sub_region_1'] != 'Western Australia']
mob_cols = [c for c in wa_data.columns if '_percent_change_from_baseline' in c]
wa_mobility = wa_data[mob_cols]
mob_locations = [c.replace('_percent_change_from_baseline', '') for c in wa_mobility.columns]
wa_mobility.columns = mob_locations
non_wa_mobility = non_wa_data[['sub_region_1'] + mob_cols]
non_wa_mobility.columns = ['sub_region_1'] + mob_locations
wa_mobility.index = pd.to_datetime(wa_mobility.index)
non_wa_mobility.index = pd.to_datetime(non_wa_mobility.index)
non_wa_mobility['fake_weights'] = 1.0
state_averages = pd.DataFrame(columns=mob_locations)
for location in mob_locations:
    state_averages[location] = non_wa_mobility.groupby(non_wa_mobility.index).apply(lambda x: np.average(x[location], weights=x['fake_weights']))
non_wa_relmob = 1.0 + state_averages * 1e-2
wa_relmob = 1.0 + wa_mobility * 1e-2